In [1]:
import json
from requests_html import HTMLSession, HTML
session = HTMLSession()
from selenium import webdriver
#browser = webdriver.Chrome(executable_path='./chromedriver/chromedriver')

1. Get link_products

In [ ]:
browser = webdriver.Chrome(executable_path='./chromedriver/chromedriver')

browser.get('https://www.amazon.com/s?i=specialty-aps&bbn=16225007011&rh=n%3A16225007011%2Cn%3A13896617011&ref=nav_em_T1_0_4_NaN_3__nav_desktop_sa_intl_computers_tablets')

url = 'https://www.amazon.com/s?i=specialty-aps&bbn=16225007011&rh=n%3A16225007011%2Cn%3A13896617011&ref=nav_em_T1_0_4_NaN_3__nav_desktop_sa_intl_computers_tablets'

f = open("../../dataset/Amazon/amazon_product_links.txt",'w')
count = 0
while True:
    print(url)
    count += 1
    print(count)
    browser.get(url)
    post = HTML(html=browser.page_source)
    head_ = post.find('.s-include-content-margin')
    for head in head_:
        pos = head.find('.a-text-normal')
        if len(pos) != 0:
            url = 'https://www.amazon.com/' + pos[0].attrs['href'] + '\n'
            f.write(url)

    check_next = post.find('.a-last')
    if len(check_next) == 1:
        url = 'https://www.amazon.com/' + check_next[0].find('a')[0].attrs['href']       
    else:
        break

f.close()

2. Get attributes

In [ ]:
def getAttributes():
    res = ['Price']
    
    browser.get('https://www.amazon.com/s?i=specialty-aps&bbn=16225007011&rh=n%3A16225007011%2Cn%3A13896617011&ref=nav_em_T1_0_4_NaN_3__nav_desktop_sa_intl_computers_tablets')
    
    f = open('../../dataset/Amazon/amazon_product_links.txt', 'r')
    links = f.readlines()
    f.close()

    count = 0
    for link in links:
        count += 1
       
        url = link[:-1]
        url = url.split('\t')

        if (len(url) == 1):
            browser.get(url[0])
            post = HTML(html=browser.page_source)
            
            specifications = post.find('#productDetails_techSpec_section_1')
            if len(specifications) != 0:
                specifications_components = specifications[0].find('tr')
                for component in specifications_components:
                    data = component.text.split('\n')
                    if data[0] not in res:
                        res.append(data[0])
            else:
                continue
            specifications = post.find('#productDetails_techSpec_section_2')
            if len(specifications) != 0:
                specifications_components = specifications[0].find('tr')
                for component in specifications_components:
                    data = component.text.split('\n')
                    if data[0] not in res:
                        res.append(data[0])
            else:
                continue
        print(count)
        if count % 100 == 0:
            print(res)
    return res

browser = webdriver.Chrome(executable_path='./chromedriver/chromedriver')
list_ = getAttributes()
dic = dict(zip(list_, [""]*len(list_)))
r = json.dumps(dic)
f = open('../../dataset/Amazon/amazon_attrs.json', 'w')
f.write(r)
f.close()

3. Crawl data

In [2]:
class Laptop:
    def __init__(self, attrbs_list):
        self.data = dict(zip(attrbs_list, [""]*len(attrbs_list)))
        
    def convertDataToJsonStr(self):
        return json.dumps(self.data)

    def convertDataToStrCSV(self, url_=''):
        return '\t'.join(self.data.values()) + '\t' + url_ + '\n'
    
    def get_keys_list(self):
        return list(self.data.keys())
          
    def setValue(self, key, value):
        self.data[key] = value

In [3]:
with open('../../dataset/Amazon/amazon_attrs.json', 'r') as f:
    attrs = f.read()
attrbs_list = list(json.loads(attrs).keys())

In [7]:
#################################################
browser = webdriver.Chrome(executable_path='./chromedriver/chromedriver')

browser.get('https://www.amazon.com/s?i=specialty-aps&bbn=16225007011&rh=n%3A16225007011%2Cn%3A13896617011&ref=nav_em_T1_0_4_NaN_3__nav_desktop_sa_intl_computers_tablets')

f = open('../../dataset/Amazon/amazon_product_links.txt', 'r')
links = f.readlines()
f.close()

g = open('../../dataset/Amazon/amazon_product_infos.csv', 'w', encoding="utf-8")
g1 = open('../../dataset/Amazon/amazon_product_infos.txt', 'w', encoding="utf-8")
f = open('../../dataset/Amazon/special_links.txt', 'w')

temp = Laptop(attrbs_list)
title = '\t'.join(temp.get_keys_list()) + '\n'
g.write(title)

count = 0
for link in links:
    count += 1

    url = link[:-1]
    url = url.split('\t')
    
    if (len(url) == 1):
        laptop = Laptop(attrbs_list)
        
        browser.get(url[0])
        post = HTML(html=browser.page_source)

        specifications = post.find('#olp-upd-new')
        if len(specifications) == 0:
            specifications = post.find('#olp-upd-new-used')
            if len(specifications) == 0:
                specifications = post.find('#priceblock_ourprice')
                if len(specifications) == 0:
                    specifications = post.find('#olp-upd-new')
                    if len(specifications) == 0:
                        specifications = post.find('#olp-upd-used')
        
        if len(specifications) != 0:
            specifications = specifications[0].find('.a-color-price')
            if len(specifications) != 0:
                print('count: %d - price: %s'%(count, specifications[0].text))
                laptop.setValue('Price', specifications[0].text)
            else:
                f.write('1' + url[0] + '\n')
                print('1' + url[0])
        else:
            f.write('2' + url[0] + '\n')
            print('2' + url[0])


        specifications = post.find('#productDetails_techSpec_section_1')
        if len(specifications) != 0:
            specifications_components = specifications[0].find('tr')
            for component in specifications_components:
                data = component.text.split('\n')
                if len(data) == 2:
                    laptop.setValue(data[0], data[1])
        else:
            f.write('3' + url[0] + '\n')
            print('count: %d - not found id #productDetails_techSpec_section_1'%(count))
            continue
        specifications = post.find('#productDetails_techSpec_section_2')
        if len(specifications) != 0:
            specifications_components = specifications[0].find('tr')
            for component in specifications_components:
                data = component.text.split('\n')
                if len(data) == 2:
                    laptop.setValue(data[0], data[1])
        else:
            f.write('4' + url[0] + '\n')
            print('count: %d - not found id #productDetails_techSpec_section_2'%(count))
            continue
    
    g.write(laptop.convertDataToStrCSV(url_=url[0]))
    g1.write(laptop.convertDataToStrCSV(url_=url[0]))
    
g.close()
g1.close()
f.close()

count: 1978 - price: $1,179.90
count: 1979 - price: $409.95
count: 1980 - price: $238.99
count: 1981 - price: $279.00
count: 1982 - price: $548.60
count: 1983 - price: $1,659.00
count: 1984 - price: $981.98
count: 1985 - price: $169.95
count: 1986 - price: $686.96
count: 1987 - price: $218.99
count: 1988 - price: $519.00
count: 1989 - price: $69.00
count: 1990 - price: $184.99
count: 1991 - price: $739.00
count: 1992 - price: $329.99
count: 1992 - not found id #productDetails_techSpec_section_1
count: 1993 - price: $49.90
count: 1994 - price: $335.00
count: 1995 - price: $1,499.00
count: 1996 - price: $715.00
count: 1997 - price: $789.00
count: 1998 - price: $549.00
count: 1999 - price: $679.00
count: 2000 - price: $590.00
count: 2001 - price: $504.99
count: 2002 - price: $475.99
count: 2003 - price: $789.00
count: 2004 - price: $829.00
count: 2005 - price: $859.99
count: 2006 - price: $1,085.00
count: 2007 - price: $590.00
count: 2008 - price: $844.99
count: 2009 - price: $1,099.00
co